In [23]:
import tensorflow as tf
import numpy as np
try:
    utils
except:
    import utils
else:    
    reload(utils)
    print 'utils has been reloaded'

utils has been reloaded


In [158]:
data_set = utils.Dataset(root_dir='data/gifs_300')

In [159]:
data, target = data_set.get_random_batch(128,replace=False)

In [160]:
data_set.data.shape

(2522, 11, 4096)

np.reshape(data[:,0,:],[data.shape[0],data.shape[2]])

In [152]:
INPUT_EMBEDDING_SIZE = 4096
OUTPUT_EMBEDDING_SIZE = 4096
IS_TRAINING = True
N_HIDDEN = 128
# N_STEPS = 10
# BATCH_SIZE = 5

In [155]:
def build_model(output_embedding_size=OUTPUT_EMBEDDING_SIZE,
                input_embedding_size=INPUT_EMBEDDING_SIZE,
                is_training=IS_TRAINING,
                n_hidden=N_HIDDEN
               ):
    """
    Builds the model for the LSTM model
    :param vocab_size: Size of the vocab from the input function
    :param is_training: Boolean, true when training
    :param n_hidden: N hidden units
    :param n_units: N LSTM units
    :param embedding_size: Size of the embedding dimensions
    :param n_steps: N timesteps we train over
    :return:
           predictions: Softmax prediction over words
           loss: Loss from current batch
           x: Placeholder for inputs
           y: Placeholder for labels
    """
    with tf.name_scope("placeholders"):
        x = tf.placeholder(dtype=tf.int32, shape=[None, input_embedding_size])
        y = tf.placeholder(dtype=tf.int32, shape=[None, output_embedding_size])
        #learning_rate = tf.placeholder(tf.int32, [None])

    # Creates the actual model using embeddings as input
    with tf.name_scope("model"):
        inputs = tf.cast(x, tf.float32)
        hidden1 = tf.layers.dense(inputs, units=n_hidden, activation=tf.nn.relu)
        dropout_1 = tf.layers.dropout(hidden1, rate=0, training=is_training)
        predictions = tf.layers.dense(dropout_1, units=output_embedding_size, activation=None)
        mse = tf.losses.mean_squared_error(labels=y,
                                           predictions=predictions)
#         cde = tf.losses.cosine_distance(labels=y,
#                                         predictions=predictions)
        loss = tf.reduce_mean(mse)
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.01)
        train_step = optimizer.minimize(mse)
        
#         learning_rate = tf.train.exponential_decay(
#             0.01,                # Base learning rate.
#             batch * BATCH_SIZE,  # Current index into the dataset.
#             train_size,          # Decay step.
#             0.95,                # Decay rate.
#             staircase=True)
#         # Use simple momentum for the optimization
#         optimizer = tf.train.MomentumOptimizer(learning_rate,
#                                                0.9).minimize(loss,
#                                                              global_step=batch)
        return predictions, loss, x, y,dropout_1,train_step

In [156]:
g = tf.Graph()
with g.as_default():
    sess = tf.InteractiveSession()
    for i in range(100):
        data, _ = data_set.get_random_batch(128, replace=False)
        x_batch = np.reshape(data[:,np.random.choice(data.shape[1]),:],
                             [data.shape[0],data.shape[2]])
        y_batch = x_batch
        predictions, loss, x, y,dropout_1,train_step = build_model()    
        sess.run(tf.global_variables_initializer())
        predictions_out, loss_out,dropout_1,train_step = sess.run([predictions, loss, dropout_1, train_step],
                                             feed_dict={x: x_batch,
                                                        y: y_batch})
        if i%10 == 0:
            print(predictions_out.shape, loss_out)
        

((128, 4096), 1.0452249)
((70, 4096), 0.95786232)
((128, 4096), 0.97341549)
((128, 4096), 0.94549811)


KeyboardInterrupt: 

In [127]:
tf.layers.batch_normalization(tf.cast(x_batch, tf.float32))

<tf.Tensor 'batch_normalization_3/batchnorm/add_1:0' shape=(70, 4096) dtype=float32>

In [145]:
loss_out

0.94178534

In [139]:
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.
